In [1]:
import os
import pandas as pd
import numpy as np
import json
import pprint
import re
import nltk
import csv
import sys

import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
data_dir = "data"
data_folders = os.listdir(data_dir)

for data_folder in data_folders:
    print(data_folder,'\n',os.listdir(os.path.join(data_dir,data_folder)))
    print('-'*30)

empatheticdialogues 
 ['test.csv', 'train.csv', 'valid.csv']
------------------------------
goemotions 
 ['goemotions_1.csv', 'goemotions_2.csv', 'goemotions_3.csv']
------------------------------
processed 
 ['ed_for_goemotions.tsv', 'all_dd_data.tsv', 'all_meld_data.tsv', 'all_names.csv', 'dd_for_goemotions.tsv', 'meld_for_goemotions.tsv', 'dd_for_goemotions.tsv.eval_results.txt', 'dd_for_goemotions.tsv.label_predictions.tsv', 'dd_for_goemotions.tsv.predictions.tsv', 'dd_for_goemotions.tsv.tf_record', 'meld_for_goemotions.tsv.eval_results.txt', 'meld_for_goemotions.tsv.label_predictions.tsv', 'meld_for_goemotions.tsv.predictions.tsv', 'meld_for_goemotions.tsv.tf_record', 'emotions.csv', 'dd_goemotions_pred_broad.tsv', 'dd_goemotions_pred_broad_simple.tsv', 'meld_goemotions_pred_broad_simple.tsv', 'ed_train_clean.csv', 'ed_test_clean.csv', 'ed_valid_clean.csv', 'ed_goemotions_pred_broad_simple.tsv', 'all_ed_data.tsv', 'ed_for_goemotions.tsv.eval_results.txt', 'ed_for_goemotions.tsv.la

In [3]:
emotion_codes = ("neutral", "anger", "disgust", "fear", "joy", "sadness", "surprise")

In [4]:
data_folder='processed'
file_name='emotions.csv'
emotions = pd.read_csv(os.path.join(data_dir, data_folder, file_name))
print(emotions.shape)

emotions

(28, 6)


,fine_emo,fine_emo_id,broad_emo,broad_emo_id,sentiment,sentiment_id
0,admiration,0,joy,4,positive,3
1,amusement,1,joy,4,positive,3
2,anger,2,anger,1,negative,1
3,annoyance,3,anger,1,negative,1
4,approval,4,joy,4,positive,3
5,caring,5,joy,4,positive,3
6,confusion,6,surprise,6,ambiguous,2
7,curiosity,7,surprise,6,ambiguous,2
8,desire,8,joy,4,positive,3
9,disappointment,9,sadness,5,negative,1


### DD Data Pairing

In [5]:
file_name='dd_goemotions_pred_broad_simple.tsv'
dd_data = pd.read_csv(os.path.join(data_dir, data_folder, file_name), sep='\t')
print(dd_data.shape)
dd_data.head()

(102979, 5)


,utterance,label,id,pred,pred_broad_id
0,The kitchen stinks.,2,1_1,neutral,0
1,I 'll throw out the garbage.,0,1_2,neutral,0
2,"So [NAME], how about getting some coffee for t...",4,2_1,curiosity,6
3,Coffee? I don't honestly like that kind of stuff.,2,2_2,curiosity,6
4,"Come on, you can at least try a little, beside...",0,2_3,neutral,0


In [6]:
file_name='all_dd_data.tsv'
all_dd_data = pd.read_csv(os.path.join(data_dir, data_folder, file_name), sep='\t')
print(all_dd_data.shape)
all_dd_data.head()

(102979, 6)


,convo_idx,topic,utterance_idx,utterance,act_label,emotion_label
0,1,1,1,The kitchen stinks.,3,2
1,1,1,2,I 'll throw out the garbage.,4,0
2,2,1,1,"So [NAME], how about getting some coffee for t...",3,4
3,2,1,2,Coffee? I don't honestly like that kind of stuff.,4,2
4,2,1,3,"Come on, you can at least try a little, beside...",3,0


In [7]:
dd_data["convo_id"] = all_dd_data["convo_idx"]
dd_data["utterance_id"] = all_dd_data["utterance_idx"]

dd_data.head()

,utterance,label,id,pred,pred_broad_id,convo_id,utterance_id
0,The kitchen stinks.,2,1_1,neutral,0,1,1
1,I 'll throw out the garbage.,0,1_2,neutral,0,1,2
2,"So [NAME], how about getting some coffee for t...",4,2_1,curiosity,6,2,1
3,Coffee? I don't honestly like that kind of stuff.,2,2_2,curiosity,6,2,2
4,"Come on, you can at least try a little, beside...",0,2_3,neutral,0,2,3


In [8]:
## For each utterance, get the response (next utterance) data
dd_data["next_utter_id"] = dd_data["utterance_id"].shift(-1).fillna(0).astype(int)
dd_data["next_utter"] = dd_data["utterance"].shift(-1).fillna("")
dd_data["next_utter_label"] = dd_data["label"].shift(-1).fillna(-1).astype(int)
dd_data["next_utter_pred"] = dd_data["pred"].shift(-1).fillna("end_convo")

## Make sure end of conversations are properly labelled
dd_data['end_convo'] = dd_data['next_utter_id'] < dd_data['utterance_id']
dd_data.loc[dd_data['end_convo'] == True, 'next_utter'] = ""
dd_data.loc[dd_data['end_convo'] == True, 'next_utter_id'] = 0
dd_data.loc[dd_data['end_convo'] == True, 'next_utter_label'] = -1
dd_data.loc[dd_data['end_convo'] == True, 'next_utter_pred'] = "end_convo"

dd_data

,utterance,label,id,pred,pred_broad_id,convo_id,utterance_id,next_utter_id,next_utter,next_utter_label,next_utter_pred,end_convo
0,The kitchen stinks.,2,1_1,neutral,0,1,1,2,I 'll throw out the garbage.,0,neutral,False
1,I 'll throw out the garbage.,0,1_2,neutral,0,1,2,0,,-1,end_convo,True
2,"So [NAME], how about getting some coffee for t...",4,2_1,curiosity,6,2,1,2,Coffee? I don't honestly like that kind of stuff.,2,curiosity,False
3,Coffee? I don't honestly like that kind of stuff.,2,2_2,curiosity,6,2,2,3,"Come on, you can at least try a little, beside...",0,neutral,False
4,"Come on, you can at least try a little, beside...",0,2_3,neutral,0,2,3,4,What's wrong with that? Cigarette is the thing...,1,curiosity,False
...,...,...,...,...,...,...,...,...,...,...,...,...
102974,"Well, thank you very much for all that informa...",0,13118_11,gratitude,4,13118,11,12,Are you going to make an offer today?,0,curiosity,False
102975,Are you going to make an offer today?,0,13118_12,curiosity,6,13118,12,13,Yes. My customer is in urgent need of the stee...,0,neutral,False
102976,Yes. My customer is in urgent need of the stee...,0,13118_13,neutral,0,13118,13,14,"Ok, I 'll get this rate right away.",0,optimism,False
102977,"Ok, I 'll get this rate right away.",0,13118_14,optimism,4,13118,14,15,Thank you.,4,gratitude,False


In [9]:
## Get label counts for emotional responses
dd_pairs_broad = dd_data.groupby(['label', 'next_utter_label']).agg({'next_utter_label':'count'}).unstack().fillna(0)
dd_pairs_broad = dd_pairs_broad.astype(int)

dd_pairs_broad.columns = ["end_convo", "neutral", "anger", "disgust", "fear", "joy", "sadness", "surprise"]
dd_pairs_broad.index = ["neutral", "anger", "disgust", "fear", "joy", "sadness", "surprise"]

dd_pairs_broad

,end_convo,neutral,anger,disgust,fear,joy,sadness,surprise
neutral,8970,67606,677,223,125,5683,887,1401
anger,156,686,108,3,1,11,40,17
disgust,51,238,5,42,0,5,2,10
fear,17,136,0,0,5,8,0,8
joy,3578,3196,8,11,7,5898,18,169
sadness,172,789,34,1,1,45,65,43
surprise,174,1346,23,11,6,179,24,60


In [10]:
row_sums = dd_pairs_broad.sum(axis=1)
row_sums

neutral     85572
anger        1022
disgust       353
fear          174
joy         12885
sadness      1150
surprise     1823
dtype: int64

In [11]:
pct = dd_pairs_broad.div(row_sums, axis=0)
pct

,end_convo,neutral,anger,disgust,fear,joy,sadness,surprise
neutral,0.104824,0.790048,0.007911,0.002606,0.001461,0.066412,0.010366,0.016372
anger,0.152642,0.671233,0.105675,0.002935,0.000978,0.010763,0.039139,0.016634
disgust,0.144476,0.674221,0.014164,0.118980,0.000000,0.014164,0.005666,0.028329
fear,0.097701,0.781609,0.000000,0.000000,0.028736,0.045977,0.000000,0.045977
joy,0.277687,0.248040,0.000621,0.000854,0.000543,0.457742,0.001397,0.013116
sadness,0.149565,0.686087,0.029565,0.000870,0.000870,0.039130,0.056522,0.037391
surprise,0.095447,0.738343,0.012617,0.006034,0.003291,0.098190,0.013165,0.032913


In [12]:
## Get label counts for emotional responses
dd_pairs_fine = dd_data.groupby(['pred', 'next_utter_pred']).agg({'next_utter_pred':'count'}).unstack().fillna(0)
dd_pairs_fine = dd_pairs_fine.astype(int)

dd_pairs_fine

next_utter_pred                                               \
next_utter_pred      admiration amusement anger annoyance approval confusion   
pred                                                                           
admiration                  699        33     6         1      239         4   
amusement                    31        21     2         0       15         0   
anger                         4         0     0         0        7         1   
annoyance                     0         0     0         0        0         0   
approval                    251        14     4         0      105         2   
confusion                     8         0     0         0        4         0   
curiosity                  1569        93    20         1     1086        61   
desire                       46         3     1         0       25         1   
excitement                    1         0     0         0        1         0   
gratitude                   240         5     3         0       72         5   
joy                         139         6     0         0       59         0   
love                        152         7     2         0       40         2   
neutral                    2964       139    77         7     1543        50   
optimism                    127         3     2         0       45         3   
remorse                      43         4     2         1       56         1   
sadness                      39         6     1         0       39         2   
surprise                     50         6     0         0       35         1   

                                                                           \
next_utter_pred curiosity desire end_convo excitement gratitude  joy love   
pred                                                                        
admiration           1245     32      1159          1       401  161  106   
amusement              60      1        53          0         7   13    8   
anger                  27      0        24          0         2    0    2   
annoyance               2      0         1          0         0    0    1   
approval              829      9       687          2       171   43   37   
confusion              32      0        13          0         2    6    2   
curiosity            2969    315       484         10       481  349  492   
desire                264      2        52          0        11    7    6   
excitement              6      2         8          0         0    1    5   
gratitude             467     10      1859          1       504   81   33   
joy                   274      9       292          2       122   76   20   
love                  364      2       147          0        33   24   63   
neutral             12272    243      7203         15      2245  540  453   
optimism              242      9       686          1       271   29   17   
remorse               305      6       229          0        88   10    8   
sadness               286      2       132          1        30   10    5   
surprise              133      2        89          0         9   14   14   

                                                           
next_utter_pred neutral optimism remorse sadness surprise  
pred                                                       
admiration         2591      163      43      34       41  
amusement           132        4       6       3        1  
anger                71        4       5       5        1  
annoyance             5        0       1       0        0  
approval           1204       93      45      22       19  
confusion            76        6       2       2        1  
curiosity         16598      380     349     333      117  
desire              335       28      12       4        1  
excitement           13        0       0       0        0  
gratitude          1119      143      27      13        9  
joy                 481       29      24      14        7  
love                511       25  

In [13]:
## Play around with different emotion pairings
selected = dd_data[dd_data["label"] == 5]
selected = selected[selected["next_utter_label"].isin([5])]
selected

,utterance,label,id,pred,pred_broad_id,convo_id,utterance_id,next_utter_id,next_utter,next_utter_label,next_utter_pred,end_convo
1177,But that still sounds expensive.,5,251_9,neutral,0,251,9,10,"Sorry, we can't come down any more.",5,remorse,False
1424,Ouch! I just bit my tongue.,5,288_1,neutral,0,288,1,2,"Oh, that hurts.",5,sadness,False
1425,"Oh, that hurts.",5,288_2,sadness,5,288,2,3,I think it 's bleeding.,5,neutral,False
1568,Thank you. But a number of things have been go...,5,309_2,gratitude,4,309,2,3,That 's too bad. What problems are you having.,5,neutral,False
2234,I 'm afraid that 's the best we can do.,5,382_11,sadness,5,382,11,12,"If so, I 'd better try another store.",5,approval,False
...,...,...,...,...,...,...,...,...,...,...,...,...
91303,"No, that would be crazy. But then who will it ...",5,11727_9,curiosity,6,11727,9,10,"Well, it could be me too. But I am not too wor...",5,neutral,False
94907,"Oh, that 's terrible! What bad luck! I wish we...",5,12196_7,optimism,4,12196,7,8,"I 'm sorry, sir.",5,remorse,False
94919,"Oh, this is terrible, Yuri. Terrible! So we ca...",5,12196_19,confusion,6,12196,19,20,"I 'm sorry, sir.",5,remorse,False
94920,"I 'm sorry, sir.",5,12196_20,remorse,5,12196,20,21,"You do n't have to keep saying that, Yuri. It ...",5,remorse,False


### ED Data Pairing

In [14]:
file_name='ed_goemotions_pred_broad_simple.tsv'
ed_data = pd.read_csv(os.path.join(data_dir, data_folder, file_name), sep='\t')
print(ed_data.shape)
ed_data.head()

(84169, 5)


,utterance,label,id,pred,pred_broad_id
0,I remember going to see the fireworks with my ...,0,1_1,0,4
1,"Was this a friend you were in love with, or ju...",0,1_2,18,4
2,This was a best friend. I miss her.,0,1_3,4,4
3,Where has she gone?,0,1_4,7,6
4,We no longer talk.,0,1_5,27,0


In [15]:
file_name='all_ed_data.tsv'
all_ed_data = pd.read_csv(os.path.join(data_dir, data_folder, file_name), sep='\t')
print(all_ed_data.shape)
all_ed_data.head()

(84169, 9)


,conv_id,utterance_idx,context,prompt,speaker_idx,utterance,selfeval,tags,conv_num
0,hit:0_conv:1,1,sentimental,I remember going to the fireworks with my best...,1,I remember going to see the fireworks with my ...,5|5|5_2|2|5,NaN,1
1,hit:0_conv:1,2,sentimental,I remember going to the fireworks with my best...,0,Was this a friend you were in love with_comma_...,5|5|5_2|2|5,NaN,1
2,hit:0_conv:1,3,sentimental,I remember going to the fireworks with my best...,1,This was a best friend. I miss her.,5|5|5_2|2|5,NaN,1
3,hit:0_conv:1,4,sentimental,I remember going to the fireworks with my best...,0,Where has she gone?,5|5|5_2|2|5,NaN,1
4,hit:0_conv:1,5,sentimental,I remember going to the fireworks with my best...,1,We no longer talk.,5|5|5_2|2|5,NaN,1


In [16]:
ed_data["convo_id"] = all_ed_data["conv_num"]
ed_data["utterance_id"] = all_ed_data["utterance_idx"]

ed_data.head()

,utterance,label,id,pred,pred_broad_id,convo_id,utterance_id
0,I remember going to see the fireworks with my ...,0,1_1,0,4,1,1
1,"Was this a friend you were in love with, or ju...",0,1_2,18,4,1,2
2,This was a best friend. I miss her.,0,1_3,4,4,1,3
3,Where has she gone?,0,1_4,7,6,1,4
4,We no longer talk.,0,1_5,27,0,1,5


In [17]:
## For each utterance, get the response (next utterance) data
ed_data["next_utter_id"] = ed_data["utterance_id"].shift(-1).fillna(0).astype(int)
ed_data["next_utter"] = ed_data["utterance"].shift(-1).fillna("")
ed_data["next_utter_broad"] = ed_data["pred_broad_id"].shift(-1).fillna(-1).astype(int)
ed_data["next_utter_pred"] = ed_data["pred"].shift(-1).fillna(-1).astype(int)

## Make sure end of conversations are properly labelled
ed_data['end_convo'] = ed_data['next_utter_id'] < ed_data['utterance_id']
ed_data.loc[ed_data['end_convo'] == True, 'next_utter'] = ""
ed_data.loc[ed_data['end_convo'] == True, 'next_utter_id'] = 0
ed_data.loc[ed_data['end_convo'] == True, 'next_utter_broad'] = -1
ed_data.loc[ed_data['end_convo'] == True, 'next_utter_pred'] = -1

ed_data

,utterance,label,id,pred,pred_broad_id,convo_id,utterance_id,next_utter_id,next_utter,next_utter_broad,next_utter_pred,end_convo
0,I remember going to see the fireworks with my ...,0,1_1,0,4,1,1,2,"Was this a friend you were in love with, or ju...",4,18,False
1,"Was this a friend you were in love with, or ju...",0,1_2,18,4,1,2,3,This was a best friend. I miss her.,4,4,False
2,This was a best friend. I miss her.,0,1_3,4,4,1,3,4,Where has she gone?,6,7,False
3,Where has she gone?,0,1_4,7,6,1,4,5,We no longer talk.,0,27,False
4,We no longer talk.,0,1_5,27,0,1,5,6,Oh was this something that happened because of...,6,7,False
...,...,...,...,...,...,...,...,...,...,...,...,...
84164,Yeah reminds me of the good old days. I miss m...,0,24848_5,27,0,24848,5,0,,-1,-1,True
84165,I woke up this morning to my wife telling me s...,0,24849_1,17,4,24849,1,2,Oh hey that's awesome! That is awesome right?,4,0,False
84166,Oh hey that's awesome! That is awesome right?,0,24849_2,0,4,24849,2,3,It is soooo awesome. We have been wanting a ba...,4,0,False
84167,It is soooo awesome. We have been wanting a ba...,0,24849_3,0,4,24849,3,4,That is awesome!!!! Congratulations!,4,0,False


In [18]:
ed_data.pred_broad_id.value_counts()

0    34543
4    31907
6    11369
5     5866
1      484
Name: pred_broad_id, dtype: int64

In [19]:
## Get broad emotion counts for emotional responses
ed_pairs = ed_data.groupby(['pred_broad_id', 'next_utter_broad']).agg({'next_utter_broad':'count'}).unstack().fillna(0)
ed_pairs = ed_pairs.astype(int)

 ## nothing predicted for disgust and fear
ed_pairs.columns = ["end_convo", "neutral", "anger", "joy", "sadness", "surprise"]
ed_pairs.index = ["neutral", "anger", "joy", "sadness", "surprise"]

ed_pairs

,end_convo,neutral,anger,joy,sadness,surprise
neutral,6955,9690,190,9759,1935,6014
anger,70,212,12,90,34,66
joy,9475,6695,42,12099,693,2903
sadness,974,1810,35,1311,744,992
surprise,2057,5429,52,2654,543,634


In [20]:
## Play around with different emotion pairings
selected = ed_data[ed_data["pred_broad_id"] == 0]
selected = selected[selected["next_utter_broad"].isin([0])]
selected

,utterance,label,id,pred,pred_broad_id,convo_id,utterance_id,next_utter_id,next_utter,next_utter_broad,next_utter_pred,end_convo
9,I do actually hit blank walls a lot of times b...,0,2_4,27,0,2,4,5,i virtually thought so.. and i used to get sw...,0,27,False
17,I have never cheated on my wife.,0,4_1,27,0,4,1,2,"And thats something you should never do, good ...",0,27,False
21,"Job interviews always make me sweat bullets, m...",0,5_1,27,0,5,1,2,Don't be nervous. Just be prepared.,0,27,False
22,Don't be nervous. Just be prepared.,0,5_2,27,0,5,2,3,I feel like getting prepared and then having a...,0,27,False
23,I feel like getting prepared and then having a...,0,5_3,27,0,5,3,4,Yes but if you stay calm it will be ok.,0,27,False
...,...,...,...,...,...,...,...,...,...,...,...,...
84096,Ugh that must so inconvenient and uncomfortable!,0,24830_2,27,0,24830,2,3,"It really is, I just have social anxiety reall...",0,27,False
84097,"It really is, I just have social anxiety reall...",0,24830_3,27,0,24830,3,4,"I can relate, I get red face when ever I have ...",0,27,False
84130,It's a racket. The building is from the 1950s ...,0,24839_3,27,0,24839,3,4,"Mine hadn't been replaced in 6 years, I found ...",0,27,False
84136,People can give me all the bullshit advice the...,0,24841_1,27,0,24841,1,2,Good on you. You need to be comfortable in you...,0,27,False


In [21]:
## Get broad emotion counts for emotional responses
ed_pairs = ed_data.groupby(['pred', 'next_utter_pred']).agg({'next_utter_pred':'count'}).unstack().fillna(0)
ed_pairs = ed_pairs.astype(int)

## Get the word labels
ed_pairs.columns = ["end_convo"] + [emotions.fine_emo[x] for x in emotions.index.tolist() if x in ed_pairs.index.tolist()]
ed_pairs.index = [emotions.fine_emo[x] for x in emotions.index.tolist() if x in ed_pairs.index.tolist()]

ed_pairs

,end_convo,admiration,amusement,anger,annoyance,approval,confusion,curiosity,desire,excitement,gratitude,joy,love,optimism,remorse,sadness,surprise,neutral
admiration,3146,1905,307,7,1,541,7,682,76,13,413,507,315,399,68,113,143,2499
amusement,964,224,274,3,1,96,3,141,18,2,16,85,53,103,36,60,32,691
anger,62,10,16,11,0,22,3,51,2,0,4,8,7,14,14,17,5,200
annoyance,8,0,3,1,0,1,0,7,1,0,0,1,0,1,2,1,0,12
approval,1027,777,128,10,0,156,15,583,22,3,94,171,92,321,84,73,94,990
confusion,37,6,5,1,1,9,1,12,1,0,2,3,4,12,3,19,2,122
curiosity,1660,554,285,40,1,464,17,227,61,7,68,217,152,172,67,384,94,4755
desire,167,62,22,1,0,27,1,77,12,0,8,13,11,52,19,11,3,198
excitement,18,19,5,0,0,2,0,30,0,0,1,4,3,6,0,1,4,28
gratitude,630,227,42,3,0,78,1,101,13,0,96,85,30,136,14,13,13,386


In [22]:
reference = pd.DataFrame(ed_data["pred"].value_counts())
reference["emotion"] = [emotions.fine_emo[x] for x in reference.index.tolist() if x in emotions.index.tolist()]
reference.sort_values(by="emotion")

,pred,emotion
0,11142,admiration
1,2802,amusement
2,446,anger
3,38,annoyance
4,4640,approval
6,240,confusion
7,9225,curiosity
8,684,desire
13,121,excitement
15,1868,gratitude


In [23]:
## Play around
selected = ed_data[ed_data["pred"] == 27]
selected = selected[selected["next_utter_pred"].isin([27])]
selected

,utterance,label,id,pred,pred_broad_id,convo_id,utterance_id,next_utter_id,next_utter,next_utter_broad,next_utter_pred,end_convo
9,I do actually hit blank walls a lot of times b...,0,2_4,27,0,2,4,5,i virtually thought so.. and i used to get sw...,0,27,False
17,I have never cheated on my wife.,0,4_1,27,0,4,1,2,"And thats something you should never do, good ...",0,27,False
21,"Job interviews always make me sweat bullets, m...",0,5_1,27,0,5,1,2,Don't be nervous. Just be prepared.,0,27,False
22,Don't be nervous. Just be prepared.,0,5_2,27,0,5,2,3,I feel like getting prepared and then having a...,0,27,False
23,I feel like getting prepared and then having a...,0,5_3,27,0,5,3,4,Yes but if you stay calm it will be ok.,0,27,False
...,...,...,...,...,...,...,...,...,...,...,...,...
84096,Ugh that must so inconvenient and uncomfortable!,0,24830_2,27,0,24830,2,3,"It really is, I just have social anxiety reall...",0,27,False
84097,"It really is, I just have social anxiety reall...",0,24830_3,27,0,24830,3,4,"I can relate, I get red face when ever I have ...",0,27,False
84130,It's a racket. The building is from the 1950s ...,0,24839_3,27,0,24839,3,4,"Mine hadn't been replaced in 6 years, I found ...",0,27,False
84136,People can give me all the bullshit advice the...,0,24841_1,27,0,24841,1,2,Good on you. You need to be comfortable in you...,0,27,False


#### Filter out ED data that has both parties surprised (surprised/confused/curious) or conversations that end with one party being surprised (surprised/confused/curious)

In [24]:
file_name='ed_train_clean.csv'
ed_train = pd.read_csv(os.path.join(data_dir, data_folder, file_name))
print(ed_train.shape)

ed_train.head()

(84169, 8)


,conv_id,utterance_idx,context,prompt,speaker_idx,utterance,selfeval,tags
0,hit:0_conv:1,1,sentimental,I remember going to the fireworks with my best...,1,I remember going to see the fireworks with my ...,5|5|5_2|2|5,NaN
1,hit:0_conv:1,2,sentimental,I remember going to the fireworks with my best...,0,Was this a friend you were in love with_comma_...,5|5|5_2|2|5,NaN
2,hit:0_conv:1,3,sentimental,I remember going to the fireworks with my best...,1,This was a best friend. I miss her.,5|5|5_2|2|5,NaN
3,hit:0_conv:1,4,sentimental,I remember going to the fireworks with my best...,0,Where has she gone?,5|5|5_2|2|5,NaN
4,hit:0_conv:1,5,sentimental,I remember going to the fireworks with my best...,1,We no longer talk.,5|5|5_2|2|5,NaN


In [25]:
ed_train["conv_num"] = ed_data["convo_id"]
ed_train.head(10)

,conv_id,utterance_idx,context,prompt,speaker_idx,utterance,selfeval,tags,conv_num
0,hit:0_conv:1,1,sentimental,I remember going to the fireworks with my best...,1,I remember going to see the fireworks with my ...,5|5|5_2|2|5,NaN,1
1,hit:0_conv:1,2,sentimental,I remember going to the fireworks with my best...,0,Was this a friend you were in love with_comma_...,5|5|5_2|2|5,NaN,1
2,hit:0_conv:1,3,sentimental,I remember going to the fireworks with my best...,1,This was a best friend. I miss her.,5|5|5_2|2|5,NaN,1
3,hit:0_conv:1,4,sentimental,I remember going to the fireworks with my best...,0,Where has she gone?,5|5|5_2|2|5,NaN,1
4,hit:0_conv:1,5,sentimental,I remember going to the fireworks with my best...,1,We no longer talk.,5|5|5_2|2|5,NaN,1
5,hit:0_conv:1,6,sentimental,I remember going to the fireworks with my best...,0,Oh was this something that happened because of...,5|5|5_2|2|5,NaN,1
6,hit:1_conv:2,1,afraid,i used to scare for darkness,2,it feels like hitting to blank wall when i se...,4|3|4_3|5|5,NaN,2
7,hit:1_conv:2,2,afraid,i used to scare for darkness,3,Oh ya? I don't really see how,4|3|4_3|5|5,NaN,2
8,hit:1_conv:2,3,afraid,i used to scare for darkness,2,dont you feel so.. its a wonder,4|3|4_3|5|5,NaN,2
9,hit:1_conv:2,4,afraid,i used to scare for darkness,3,I do actually hit blank walls a lot of times b...,4|3|4_3|5|5,NaN,2


In [26]:
## Using broad emotion labels
## Cut down conversations for which both parties are confused/surprised/curious, or end with one party that way
selected1 = ed_data[ed_data["pred_broad_id"] == 6]
selected1 = selected1[selected1["next_utter_broad"].isin([-1, 6])]

selected = pd.concat([selected1])

## Get the convos to cut
cut_list = selected["convo_id"].unique().tolist()
len(cut_list)

2495

In [27]:
ed_train_reduced = ed_train[ed_train["conv_num"].isin(cut_list) == False]
ed_train_reduced

,conv_id,utterance_idx,context,prompt,speaker_idx,utterance,selfeval,tags,conv_num
12,hit:1_conv:3,1,proud,I showed a guy how to run a good bead in weldi...,3,Hi how are you doing today,3|5|5_4|3|4,NaN,3
13,hit:1_conv:3,2,proud,I showed a guy how to run a good bead in weldi...,2,doing good.. how about you,3|5|5_4|3|4,NaN,3
14,hit:1_conv:3,3,proud,I showed a guy how to run a good bead in weldi...,3,Im good_comma_ trying to understand how someon...,3|5|5_4|3|4,NaN,3
15,hit:1_conv:3,4,proud,I showed a guy how to run a good bead in weldi...,2,it's quite strange that you didnt imagine it,3|5|5_4|3|4,NaN,3
16,hit:1_conv:3,5,proud,I showed a guy how to run a good bead in weldi...,3,i dont imagine feeling a lot_comma_ maybe your...,3|5|5_4|3|4,NaN,3
...,...,...,...,...,...,...,...,...,...
84164,hit:12424_conv:24848,5,sentimental,I found some pictures of my grandma in the att...,389,Yeah reminds me of the good old days. I miss ...,5|5|5_5|5|5,NaN,24848
84165,hit:12424_conv:24849,1,surprised,I woke up this morning to my wife telling me s...,294,I woke up this morning to my wife telling me s...,5|5|5_5|5|5,NaN,24849
84166,hit:12424_conv:24849,2,surprised,I woke up this morning to my wife telling me s...,389,Oh hey that's awesome! That is awesome right?,5|5|5_5|5|5,NaN,24849
84167,hit:12424_conv:24849,3,surprised,I woke up this morning to my wife telling me s...,294,It is soooo awesome. We have been wanting a b...,5|5|5_5|5|5,NaN,24849


In [28]:
print("Reduced train set % of original:", len(ed_train_reduced) / len(ed_train))

Reduced train set % of original: 0.8753460300110492


In [29]:
## Write this out to a file for reference later
ed_train_reduced.drop(["conv_num"], axis=1)
ed_train_reduced.to_csv ('data/processed/ed_train_reduced1.csv', index = False, header=True)

#### Alternatively, also cut out ones that have consecutive "neutral/neutral" back and forth responses

In [30]:
## Using broad emotion labels
## Cut down conversations for which both parties are confused/surprised/curious, or end with one party that way
selected1 = ed_data[ed_data["pred_broad_id"] == 6]
selected1 = selected1[selected1["next_utter_broad"].isin([-1, 6])]

selected2 = ed_data[ed_data["pred_broad_id"] == 0]
selected2 = selected2[selected2["next_utter_broad"].isin([0])]

selected = pd.concat([selected1, selected2])

## Get the convos to cut
cut_list = selected["convo_id"].unique().tolist()
len(cut_list)

8401

In [31]:
ed_train_reduced = ed_train[ed_train["conv_num"].isin(cut_list) == False]
ed_train_reduced

,conv_id,utterance_idx,context,prompt,speaker_idx,utterance,selfeval,tags,conv_num
12,hit:1_conv:3,1,proud,I showed a guy how to run a good bead in weldi...,3,Hi how are you doing today,3|5|5_4|3|4,NaN,3
13,hit:1_conv:3,2,proud,I showed a guy how to run a good bead in weldi...,2,doing good.. how about you,3|5|5_4|3|4,NaN,3
14,hit:1_conv:3,3,proud,I showed a guy how to run a good bead in weldi...,3,Im good_comma_ trying to understand how someon...,3|5|5_4|3|4,NaN,3
15,hit:1_conv:3,4,proud,I showed a guy how to run a good bead in weldi...,2,it's quite strange that you didnt imagine it,3|5|5_4|3|4,NaN,3
16,hit:1_conv:3,5,proud,I showed a guy how to run a good bead in weldi...,3,i dont imagine feeling a lot_comma_ maybe your...,3|5|5_4|3|4,NaN,3
...,...,...,...,...,...,...,...,...,...
84164,hit:12424_conv:24848,5,sentimental,I found some pictures of my grandma in the att...,389,Yeah reminds me of the good old days. I miss ...,5|5|5_5|5|5,NaN,24848
84165,hit:12424_conv:24849,1,surprised,I woke up this morning to my wife telling me s...,294,I woke up this morning to my wife telling me s...,5|5|5_5|5|5,NaN,24849
84166,hit:12424_conv:24849,2,surprised,I woke up this morning to my wife telling me s...,389,Oh hey that's awesome! That is awesome right?,5|5|5_5|5|5,NaN,24849
84167,hit:12424_conv:24849,3,surprised,I woke up this morning to my wife telling me s...,294,It is soooo awesome. We have been wanting a b...,5|5|5_5|5|5,NaN,24849


In [32]:
print("Reduced train set % of original:", len(ed_train_reduced) / len(ed_train))

Reduced train set % of original: 0.5670733880644893


In [33]:
## Write this out to a file for reference later
ed_train_reduced.drop(["conv_num"], axis=1)
ed_train_reduced.to_csv ('data/processed/ed_train_reduced2.csv', index = False, header=True)